In [2]:
models_to_test = [
    'Qwen/Qwen2.5-VL-3B-Instruct',
    'Qwen/Qwen2.5-VL-7B-Instruct',
    'Qwen/Qwen2.5-VL-72B-Instruct',
    'deepseek-ai/Janus-Pro-1B',
    'deepseek-ai/Janus-Pro-7B'
    ]
model = models_to_test[2]
model

'Qwen/Qwen2.5-VL-72B-Instruct'

In [12]:
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [12]:
from datasets import load_dataset

dataset = load_dataset(
  "fmb-quibdo/sergio-notebooks"
)

|  Model | Size  | GPU Memory  |  GPU  | predict time  | token len | notes    |
|---|---|---|---|---|---|---|
|Qwen2.5-VL-3B   | 7.51 GB  | 100%  | 1 L40S  |  3 min 30 sec  |  1500 |   works on 1 GPU, slow would be faster on cluster, returns complete page, no echos  |
| Qwen2.5-VL-7B  | 16.57 GB  |   |   |   |   | error with download  |
| Qwen2.5-VL-72B  |   |   |  4 L40S (out of memory) |   |   |   |

In [14]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
#model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#    "Qwen/Qwen2.5-VL-72B-Instruct", torch_dtype="auto", device_map="auto"
#)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    'Qwen/Qwen2.5-VL-72B-Instruct',
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# default processer
processor = AutoProcessor.from_pretrained(model)

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": dataset['train'][33]['image'],
            },
            {"type": "text", "text": "Extract text."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1500)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in Qwen2_5_VisionTransformerPretrainedModel is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


Loading checkpoint shards:   0%|          | 0/38 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 1 has a total capacity of 44.53 GiB of which 17.25 MiB is free. Process 23589 has 44.50 GiB memory in use. Of the allocated memory 43.99 GiB is allocated by PyTorch, and 102.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
%%writefile qwen2.5-vl-3b-text.md

'Here is the extracted text from the image:\n\n---\n\n**VENTA-ESCLAVO**\n\nEl pueblo de San Francisco de Asís, 
capital de la provincia de Catamarca, a doce días del mes de Mayo de mil ochocientos ocho: Ante mi Don José 
Dionisio de Villar Feniente de gobernador y corregidor de naturales de este dicho pueblo y de los testigos con 
quienes actúa por impedimento del único escribano, para el presente Don Juan Esteban Pinedo, vecino de esta 
provincia a quien certifico conocido y otorga que se vendrá y dará en venta real y perpetua enajenación desde ahora
y para siempre jamás a Agustín Daza vecino de esta dicha provincia que sea para el dicho sus herederos y sucesores 
como a sufrir un molato suyo propio como consta del documento simple que presento y se agrega a este protocolo 
llamado José María, el que leende por libre de otra venta, tenso, empeño, nupcialidad, facilita ni expresa y pordal
lo asegura en cantidad de diez y cuarenta pesos de plata que confiesa el otorgante tener recibidos de mano del 
comprador a su satisfacción y contento y por no ser de presente su entrega para que yo dispo teniente la 
certificación, de ella renuncia la excepción y leyes de supercargo, su prueba y demás del caso. Y confiesa el 
vendedor que los dos cientos cuarenta pesos de plata recibidos es el justo precio y verdadero valor del refeno 
muerto que no vale más, y caso de que mas valga de su demasia en época, o mucho suma te ha ce gracia, y donación al
comprador y los supos bueno, pero mera, perfecta e irreducible de las que el derecho llama intervivos con 
maniuvación en forma, sobre que renuncia la ley del ordenamiento real fecha en contes de Alcalá de Henares, y los 
cuatro años en ella declarados para repartir el engaño soy de lo que se compra vende o permite por la mitad más o 
menos de su último precio. Y desde hoy en adelante se desiste quita y aparta del derecho de patronato señorío, y 
posesión que a dicho molato tenía adquirido, y todo con las acciones reales, personales, y demás que le competen lo
cede fe- nonaria y traopasa en el comprador y quien la suya tenga para que sea su esclavo sujeto a servidumbre y 
disposición de su arbitrio mediante a que lo adquiera con extinción, obligándose a la ejecución, seguridad, 
saneamiento, y firmeza de esto venta, la cual le será cierta segura y validera, y sobre ella no se le pondrá pleito
ni contradicción alguna, y si se le moviere lo seguirá y fencera a su costa hasta dejarle\n\n---'

Writing qwen2.5-vl-3b-text.md
